In [2]:
import subprocess

In [3]:
subprocess.run('ls')

Crash test_ im_process.ipynb
Généralisation.ipynb
LICENSE
MyoQuant_He.ipynb
MyoQuant_SDH.ipynb
Préparation données.ipynb
README.md
code_calibration.ipynb
code_propre.ipynb
data
data_5_jpg.csv
data_sample_he.csv
environment.yml
images
muscle_atlas.csv
muscle_atlas_2_7.xlsx
muscle_atlas_clean.csv
notebook.ipynb
percentage cell.ipynb
recup im test.ipynb
results
sample_img
test_im5.ipynb


CompletedProcess(args='ls', returncode=0)

In [2]:
import os, sys

# Open a file
path = "/Users/afaf/stage-afaf/images"
dirs = os.listdir( path )

# This would print all the files and directories
for file in dirs:
   print(file)

30.jpg
5.jpg


In [3]:
dirs

['30.jpg', '5.jpg']

In [4]:
file_paths = []

for folder, subs, files in os.walk("/Users/afaf/stage-afaf/images"):
  for filename in files:
    file_paths.append(os.path.abspath(os.path.join(folder, filename)))

file_paths

['/Users/afaf/stage-afaf/images/30.jpg', '/Users/afaf/stage-afaf/images/5.jpg']

In [ ]:
for chemin_image in file_paths:
    myoquant= subprocess.run(['myoquant', "he-analysis", chemin_image,'--output-path',  '/Users/afaf/stage-afaf/data'])

    

In [5]:
import pandas as pd

In [6]:
df_30= pd.read_csv('data/30_results_summary.csv')
df_30

,Feature,Raw Count,Proportion (%)
0,N° Nuclei,186,100.000000
1,N° Intern. Nuclei,19,10.215054
2,N° Periph. Nuclei,152,81.720430
3,N° Cells with 1+ intern. nuc.,18,14.876033


In [118]:
df_30c=pd.read_csv('data/30_cell_details.csv')
df_30c

,label,area,centroid-0,centroid-1,eccentricity,bbox-0,bbox-1,bbox-2,bbox-3,perimeter,feret_diameter_max,N° Nuc,N° Nuc Intern,N° Nuc Periph
0,1,2187,13.569730,263.364426,0.926247,0,210,38,303,225.195959,94.921020,2,0,2
1,2,4100,13.494390,379.214878,0.982169,0,298,32,452,353.012193,156.348969,1,0,1
2,3,1092,9.039377,504.647436,0.930620,0,472,22,534,147.012193,62.393910,0,0,0
3,4,13289,50.993152,596.986229,0.606107,0,525,115,673,473.504617,160.461833,1,0,1
4,5,8730,38.030813,715.499656,0.778304,0,657,91,787,376.735065,140.890028,1,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
116,117,1484,1008.357143,223.476415,0.607228,980,202,1030,242,149.781746,52.201533,0,0,0
117,118,2529,1006.412811,42.240411,0.734286,983,10,1028,77,192.953319,68.883960,0,0,0
118,119,3018,1007.724321,502.792578,0.879805,985,461,1030,547,227.195959,86.469648,1,0,0
119,120,1321,1015.705526,1161.849357,0.767215,998,1137,1030,1189,143.740115,52.773099,0,0,0


In [119]:
fichier_details = []

for folder, subs, files in os.walk("/Users/afaf/stage-afaf/data"):
  for filename in files:
    if filename.endswith(('_cell_details.csv')):
        fichier_details.append(os.path.abspath(os.path.join(folder, filename)))

fichier_details

['/Users/afaf/stage-afaf/data/30_cell_details.csv',
 '/Users/afaf/stage-afaf/data/5_cell_details.csv']

In [120]:
df=pd.read_csv('muscle_atlas_clean.csv') #atlas du muscle
df_cell=pd.read_csv('data/30_cell_details.csv')



def conversion_pix_µm(im, data_frame,muscle_atlas):
    'Convertion du diamètre en µm'
    barre_echelle= muscle_atlas.loc[(df['Number']== im), 'Calibration (um/px)']
    
    data_frame['diameter_µm']=(data_frame['feret_diameter_max'])*barre_echelle.values

conversion_pix_µm('5.jpg',df_cell,df )

def count_atrophic_cells (data_frame,min_size, max_size):
    hyper='Hypertrophy'
    hypo= 'Hypotrophy'
    norm= 'Normal'
    data_frame["State_cell"] = ''
    for index in data_frame.index:
        a=data_frame['diameter_µm'][index]
        if a > max_size:
            data_frame["State_cell"][index]=hyper
        elif a < min_size:
            data_frame["State_cell"][index]=hypo
        else: 
            data_frame["State_cell"][index]= norm


count_atrophic_cells (df_cell, 30, 80)

df_cell

/var/folders/lc/l3h44z2d7_d98lsb5yh9d2y00000gn/T/ipykernel_43460/3172970052.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_frame["State_cell"][index]=hypo
/var/folders/lc/l3h44z2d7_d98lsb5yh9d2y00000gn/T/ipykernel_43460/3172970052.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_frame["State_cell"][index]= norm
/var/folders/lc/l3h44z2d7_d98lsb5yh9d2y00000gn/T/ipykernel_43460/3172970052.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-

,label,area,centroid-0,centroid-1,eccentricity,bbox-0,bbox-1,bbox-2,bbox-3,perimeter,feret_diameter_max,N° Nuc,N° Nuc Intern,N° Nuc Periph,diameter_µm,State_cell
0,1,2187,13.569730,263.364426,0.926247,0,210,38,303,225.195959,94.921020,2,0,2,29.615358,Hypotrophy
1,2,4100,13.494390,379.214878,0.982169,0,298,32,452,353.012193,156.348969,1,0,1,48.780878,Normal
2,3,1092,9.039377,504.647436,0.930620,0,472,22,534,147.012193,62.393910,0,0,0,19.466900,Hypotrophy
3,4,13289,50.993152,596.986229,0.606107,0,525,115,673,473.504617,160.461833,1,0,1,50.064092,Normal
4,5,8730,38.030813,715.499656,0.778304,0,657,91,787,376.735065,140.890028,1,0,1,43.957689,Normal
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
116,117,1484,1008.357143,223.476415,0.607228,980,202,1030,242,149.781746,52.201533,0,0,0,16.286878,Hypotrophy
117,118,2529,1006.412811,42.240411,0.734286,983,10,1028,77,192.953319,68.883960,0,0,0,21.491796,Hypotrophy
118,119,3018,1007.724321,502.792578,0.879805,985,461,1030,547,227.195959,86.469648,1,0,0,26.978530,Hypotrophy
119,120,1321,1015.705526,1161.849357,0.767215,998,1137,1030,1189,143.740115,52.773099,0,0,0,16.465207,Hypotrophy


In [121]:
def pourcentage_state_cell (im,data_frame,muscle_atlas):
     data = {'Number':[],'Hypertrophy': [],'Normal':[],'Hypotrophy': []}  # Ici on créer un dataframe pour mettre ensuite les valeurs des pourcentages
     df_state_cell = pd.DataFrame(data)  
     percentage= (data_frame['State_cell'].value_counts()/data_frame['State_cell'].count())*100
     df_state_cell.loc[len(df_state_cell.index)] = percentage

     
     muscle_atlas.loc[(muscle_atlas['Number']== im),"Hypertrophic_cell_%"]=df_state_cell['Hypertrophy'].values
     muscle_atlas.loc[(muscle_atlas['Number']== im),"Normal_cell_%"]=df_state_cell['Normal'].values
     muscle_atlas.loc[(muscle_atlas['Number']== im),"Hypotrophic_cell_%"]=df_state_cell['Hypotrophy'].values


pourcentage_state_cell ('30.jpg',df_cell,df)  
df.head(10)


,ordered,Number,File name,Organism,muscle,Disease Group,Diag.Final,GENE,Staining method,Age,...,Treatment,SEXE,OMIM,Mutated gene,Excluded Genes,Publication,Link,Hypertrophic_cell_%,Normal_cell_%,Hypotrophic_cell_%
0,5,5.jpg,5 10199 HE-20.jpg,Homo sapiens,deltoid,Muscular Dystrophies,Muscular Dystrophy alpha dystroglycanopathy,FKRP,HE,38y,...,none,F,607155,FKRP mutation homo Leu276Ile,NaN,NaN,NaN,NaN,NaN,NaN
1,6,6.jpg,6 10199 SDH-20.jpg,Homo sapiens,deltoid,Muscular Dystrophies,Muscular Dystrophy alpha dystroglycanopathy,FKRP,SDH,38y,...,none,F,607155,FKRP mutation homo Leu276Ile,NaN,NaN,NaN,NaN,NaN,NaN
2,13,13.jpg,13 10202 HE-20.jpg,Homo sapiens,deltoid,Muscular Dystrophies,Muscular Dystrophy alpha dystroglycanopathy,FKRP,HE,39y,...,none,F,607155,FKRP mutation homo Leu276Ile,NaN,NaN,NaN,NaN,NaN,NaN
3,14,14.jpg,14 10202 SDH-20.jpg,Homo sapiens,deltoid,Muscular Dystrophies,Muscular Dystrophy alpha dystroglycanopathy,FKRP,SDH,39y,...,none,F,607155,FKRP mutation homo Leu276Ile,NaN,NaN,NaN,NaN,NaN,NaN
4,21,21.jpg,21 10474 HE-20.jpg,Homo sapiens,quadriceps,Congenital Myopathies,NaN,ASCC1,HE,12 days,...,none,M,616867,ASCC1 mutation homo c.157dupG,ACTA1 CFL2 NEB RYR1 SMN1 TNNT1 TPM2 TPM3 RAS,J Med Genet. 2019; 56(9):617-621,https://jmg.bmj.com/content/56/9/617.long,NaN,NaN,NaN
5,30,30.jpg,30 10746 HE-10a.jpg,Homo sapiens,quadriceps,Myofibrillar Myopathies,Myofibrillar myopathy,ZASP,HE,67y,...,none,F,609452,ZASP mutation p.A165V,NaN,Acta Neuropathol. 2009;117(3):293-307.,https://link.springer.com/article/10.1007%2Fs0...,4.132231,42.14876,53.719008
6,31,31.jpg,31 10746 HE-10b.jpg,Homo sapiens,quadriceps,Myofibrillar Myopathies,Myofibrillar myopathy,ZASP,HE,67y,...,none,F,609452,ZASP mutation p.A165V,NaN,Acta Neuropathol. 2009;117(3):293-307.,https://link.springer.com/article/10.1007%2Fs0...,NaN,NaN,NaN
7,34,34.jpg,34 10746 SDH-10.jpg,Homo sapiens,quadriceps,Myofibrillar Myopathies,Myofibrillar myopathy,ZASP,SDH,67y,...,none,F,609452,ZASP mutation p.A165V,NaN,Acta Neuropathol. 2009;117(3):293-307.,https://link.springer.com/article/10.1007%2Fs0...,NaN,NaN,NaN
8,40,40.jpg,40 11015 HE-20a.jpg,Homo sapiens,deltoid,Congenital Myopathies,Centronuclear myopathy,BIN1,HE,19y,...,none,M,255200,BIN1 mutation R154Q,NaN,Neurology. 2010; 74(6):519-21.,https://n.neurology.org/content/74/6/519.long,NaN,NaN,NaN
9,41,41.jpg,41 11015 HE-20b.jpg,Homo sapiens,deltoid,Congenital Myopathies,Centronuclear myopathy,BIN1,HE,19y,...,none,M,255200,BIN1 mutation R154Q,NaN,Neurology. 2010; 74(6):519-21.,https://n.neurology.org/content/74/6/519.long,NaN,NaN,NaN


In [122]:
def diameter_mean_std (im,data_frame,muscle_atlas):
    cell_mean=data_frame['diameter_µm'].mean()
    cell_std=data_frame['diameter_µm'].std()
    muscle_atlas.loc[(muscle_atlas['Number']== im),"diameter_mean"]=cell_mean
    muscle_atlas.loc[(muscle_atlas['Number']== im),"diameter_std"]=cell_std


diameter_mean_std('30.jpg',df_cell,df)   
df.head(10)

,ordered,Number,File name,Organism,muscle,Disease Group,Diag.Final,GENE,Staining method,Age,...,OMIM,Mutated gene,Excluded Genes,Publication,Link,Hypertrophic_cell_%,Normal_cell_%,Hypotrophic_cell_%,diameter_mean,diameter_std
0,5,5.jpg,5 10199 HE-20.jpg,Homo sapiens,deltoid,Muscular Dystrophies,Muscular Dystrophy alpha dystroglycanopathy,FKRP,HE,38y,...,607155,FKRP mutation homo Leu276Ile,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,6,6.jpg,6 10199 SDH-20.jpg,Homo sapiens,deltoid,Muscular Dystrophies,Muscular Dystrophy alpha dystroglycanopathy,FKRP,SDH,38y,...,607155,FKRP mutation homo Leu276Ile,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,13,13.jpg,13 10202 HE-20.jpg,Homo sapiens,deltoid,Muscular Dystrophies,Muscular Dystrophy alpha dystroglycanopathy,FKRP,HE,39y,...,607155,FKRP mutation homo Leu276Ile,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,14,14.jpg,14 10202 SDH-20.jpg,Homo sapiens,deltoid,Muscular Dystrophies,Muscular Dystrophy alpha dystroglycanopathy,FKRP,SDH,39y,...,607155,FKRP mutation homo Leu276Ile,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,21,21.jpg,21 10474 HE-20.jpg,Homo sapiens,quadriceps,Congenital Myopathies,NaN,ASCC1,HE,12 days,...,616867,ASCC1 mutation homo c.157dupG,ACTA1 CFL2 NEB RYR1 SMN1 TNNT1 TPM2 TPM3 RAS,J Med Genet. 2019; 56(9):617-621,https://jmg.bmj.com/content/56/9/617.long,NaN,NaN,NaN,NaN,NaN
5,30,30.jpg,30 10746 HE-10a.jpg,Homo sapiens,quadriceps,Myofibrillar Myopathies,Myofibrillar myopathy,ZASP,HE,67y,...,609452,ZASP mutation p.A165V,NaN,Acta Neuropathol. 2009;117(3):293-307.,https://link.springer.com/article/10.1007%2Fs0...,4.132231,42.14876,53.719008,34.091484,19.07643
6,31,31.jpg,31 10746 HE-10b.jpg,Homo sapiens,quadriceps,Myofibrillar Myopathies,Myofibrillar myopathy,ZASP,HE,67y,...,609452,ZASP mutation p.A165V,NaN,Acta Neuropathol. 2009;117(3):293-307.,https://link.springer.com/article/10.1007%2Fs0...,NaN,NaN,NaN,NaN,NaN
7,34,34.jpg,34 10746 SDH-10.jpg,Homo sapiens,quadriceps,Myofibrillar Myopathies,Myofibrillar myopathy,ZASP,SDH,67y,...,609452,ZASP mutation p.A165V,NaN,Acta Neuropathol. 2009;117(3):293-307.,https://link.springer.com/article/10.1007%2Fs0...,NaN,NaN,NaN,NaN,NaN
8,40,40.jpg,40 11015 HE-20a.jpg,Homo sapiens,deltoid,Congenital Myopathies,Centronuclear myopathy,BIN1,HE,19y,...,255200,BIN1 mutation R154Q,NaN,Neurology. 2010; 74(6):519-21.,https://n.neurology.org/content/74/6/519.long,NaN,NaN,NaN,NaN,NaN
9,41,41.jpg,41 11015 HE-20b.jpg,Homo sapiens,deltoid,Congenital Myopathies,Centronuclear myopathy,BIN1,HE,19y,...,255200,BIN1 mutation R154Q,NaN,Neurology. 2010; 74(6):519-21.,https://n.neurology.org/content/74/6/519.long,NaN,NaN,NaN,NaN,NaN


In [123]:
fichier = []

for folder, subs, files in os.walk("/Users/afaf/stage-afaf/data"):
  for filename in files:
    if filename.endswith(('_results_summary.csv',)):
        fichier.append(os.path.abspath(os.path.join(folder, filename)))

fichier

['/Users/afaf/stage-afaf/data/5_results_summary.csv',
 '/Users/afaf/stage-afaf/data/30_results_summary.csv']

In [124]:
df_results_summary=pd.read_csv('data/30_results_summary.csv')
df3=df_results_summary.set_index('Feature')
df4=df3.transpose()
df4

Feature,N° Nuclei,N° Intern. Nuclei,N° Periph. Nuclei,N° Cells with 1+ intern. nuc.
Raw Count,186.0,19.000000,152.00000,18.000000
Proportion (%),100.0,10.215054,81.72043,14.876033


In [ ]:
#test
df.loc[(df['Number']== '30.jpg'),"Proportion_nuclei"]=df4.loc['Raw Count','N° Nuclei']
df.head(10)

In [128]:
def ajout_He (im,dataframe,muscle_atlas):
    muscle_atlas.loc[(muscle_atlas['Number']== im),"Proportion_nuclei"]=dataframe.loc['Proportion (%)','N° Nuclei']
    #muscle_atlas.loc[(muscle_atlas['Number']== im),"Number_nuclei"]=dataframe.loc['Raw count','N° Nuclei']
    muscle_atlas.loc[(muscle_atlas['Number']== im),"Number_nuclei"]=dataframe.iloc[0,0]
    muscle_atlas.loc[(muscle_atlas['Number']== im),"Proportion_intern_nuclei"]=dataframe.loc['Proportion (%)','N° Intern. Nuclei']
    #muscle_atlas.loc[(muscle_atlas['Number']== im),"Number_intern_nuclei"]=dataframe.loc['Raw Count','N° Intern. Nuclei']
    muscle_atlas.loc[(muscle_atlas['Number']== im),"Number_intern_nuclei"]=dataframe.iloc[0,1]
    #muscle_atlas.loc[(muscle_atlas['Number']== im),"Number_Periph_nuclei"]=dataframe.loc['Raw count','N° Periph. Nuclei']
    muscle_atlas.loc[(muscle_atlas['Number']== im),"Number_Periph_nuclei"]=dataframe.iloc[0,2]
    muscle_atlas.loc[(muscle_atlas['Number']== im),"Proportion_Periph_nuclei"]=dataframe.loc['Proportion (%)','N° Periph. Nuclei']
   # muscle_atlas.loc[(muscle_atlas['Number']== im),"Number_Cells with 1+ intern.nuc"]=dataframe.loc['Raw count','N° Cells with 1+ intern. nuc.']
    muscle_atlas.loc[(muscle_atlas['Number']== im),"Number_Cells with 1+ intern.nuc"]=dataframe.iloc[0,3]
    muscle_atlas.loc[(muscle_atlas['Number']== im),"Proportion_Cells with 1+ intern. nuc"]=dataframe.loc['Proportion (%)','N° Cells with 1+ intern. nuc.']

    

ajout_He('30.jpg',df4,df)   
df.head(10)

,ordered,Number,File name,Organism,muscle,Disease Group,Diag.Final,GENE,Staining method,Age,...,diameter_mean,diameter_std,Proportion_nuclei,Number_nuclei,Proportion_intern_nuclei,Number_intern_nuclei,Number_Periph_nuclei,Proportion_Periph_nuclei,Number_Cells with 1+ intern.nuc,Proportion_Cells with 1+ intern. nuc
0,5,5.jpg,5 10199 HE-20.jpg,Homo sapiens,deltoid,Muscular Dystrophies,Muscular Dystrophy alpha dystroglycanopathy,FKRP,HE,38y,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,6,6.jpg,6 10199 SDH-20.jpg,Homo sapiens,deltoid,Muscular Dystrophies,Muscular Dystrophy alpha dystroglycanopathy,FKRP,SDH,38y,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,13,13.jpg,13 10202 HE-20.jpg,Homo sapiens,deltoid,Muscular Dystrophies,Muscular Dystrophy alpha dystroglycanopathy,FKRP,HE,39y,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,14,14.jpg,14 10202 SDH-20.jpg,Homo sapiens,deltoid,Muscular Dystrophies,Muscular Dystrophy alpha dystroglycanopathy,FKRP,SDH,39y,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,21,21.jpg,21 10474 HE-20.jpg,Homo sapiens,quadriceps,Congenital Myopathies,NaN,ASCC1,HE,12 days,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,30,30.jpg,30 10746 HE-10a.jpg,Homo sapiens,quadriceps,Myofibrillar Myopathies,Myofibrillar myopathy,ZASP,HE,67y,...,34.091484,19.07643,100.0,186.0,10.215054,19.0,152.0,81.72043,18.0,14.876033
6,31,31.jpg,31 10746 HE-10b.jpg,Homo sapiens,quadriceps,Myofibrillar Myopathies,Myofibrillar myopathy,ZASP,HE,67y,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,34,34.jpg,34 10746 SDH-10.jpg,Homo sapiens,quadriceps,Myofibrillar Myopathies,Myofibrillar myopathy,ZASP,SDH,67y,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,40,40.jpg,40 11015 HE-20a.jpg,Homo sapiens,deltoid,Congenital Myopathies,Centronuclear myopathy,BIN1,HE,19y,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,41,41.jpg,41 11015 HE-20b.jpg,Homo sapiens,deltoid,Congenital Myopathies,Centronuclear myopathy,BIN1,HE,19y,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
